<a href="https://colab.research.google.com/github/tianiachan/IdentifyingAirBnBsinSantaClara/blob/main/Data670_Identifying_AirBnBs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#upload necessary csv to google drive 
from google.colab import files
uploaded = files.upload()

Saving calendar.csv to calendar.csv
Saving listings.csv to listings.csv
Saving reviews.csv to reviews.csv


In [3]:
#import as dataframe 
import pandas as pd
import io
listings_df = pd.read_csv('/content/listings.csv')
calendar_df = pd.read_csv('/content/calendar.csv')
reviews_df = pd.read_csv('/content/reviews.csv')

# Step by Step EDA in Python https://towardsdatascience.com/exploratory-data-analysis-in-python-a-step-by-step-process-d0dfa6bf94ee

In [4]:
#drop columns not needed for listings
listings_df.columns
listings_df=listings_df.drop(columns=['host_id','host_name','neighbourhood_group','price','minimum_nights','number_of_reviews','last_review','reviews_per_month','availability_365','number_of_reviews_ltm'])

In [5]:
listings_df.head()

,id,name,neighbourhood,latitude,longitude,room_type,calculated_host_listings_count,license
0,2861174.0,Casablanca Master Suite,Palo Alto,37.41710,-122.12669,Private room,1,NaN
1,1364787.0,Iris House Cozy room in dwntn Palo Alto & Stan...,Palo Alto,37.44973,-122.15081,Private room,5,NaN
2,2880487.0,**Premium Executive Condo. with Maid Svcs**,Los Altos,37.37318,-122.10736,Entire home/apt,1,NaN
3,1366049.0,Beautiful Single Family Home,Campbell,37.26901,-121.96325,Private room,1,NaN
4,4952.0,Butterfly Inn near Stanford with weekly cleaning,Palo Alto,37.43932,-122.15741,Private room,5,NaN


In [6]:
#same for reviews
reviews_df.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [7]:
#drop the columns
reviews_df=reviews_df.drop(columns=['reviewer_id'])

In [8]:
reviews_df.head()

,listing_id,id,date,reviewer_name,comments
0,4952,6560,2009-08-02,Diane,Staying with Mei Lin and Ross at the Butterfly...
1,1353178,7471795,2013-09-21,Jesus,I will definitely stay at HackerHome when I go...
2,1353178,10120145,2014-02-01,Jeanne,Rob and Vic = family.\r<br/>Class acts.
3,1353178,11096627,2014-03-20,Debora,"Very nice place, nice people, nice location! I..."
4,1353178,11875945,2014-04-17,Pascal,I had a really good time in the HackerHome. Ev...


In [9]:
#rename column id so it says review id
reviews_df.rename(columns={'id':'review_id'}, inplace=True)

In [10]:
#check that changes happened
reviews_df.head()

,listing_id,review_id,date,reviewer_name,comments
0,4952,6560,2009-08-02,Diane,Staying with Mei Lin and Ross at the Butterfly...
1,1353178,7471795,2013-09-21,Jesus,I will definitely stay at HackerHome when I go...
2,1353178,10120145,2014-02-01,Jeanne,Rob and Vic = family.\r<br/>Class acts.
3,1353178,11096627,2014-03-20,Debora,"Very nice place, nice people, nice location! I..."
4,1353178,11875945,2014-04-17,Pascal,I had a really good time in the HackerHome. Ev...


In [11]:
#do the same for calendar
calendar_df.columns

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

In [12]:
calendar_df=calendar_df.drop(columns=['price','adjusted_price','minimum_nights','maximum_nights'])

In [13]:
# calendar_df.loc[calendar_df['listing_id'] == '1364787']
# rslt_df = calendar_df.loc[calendar_df['listing_id'] == 1364787]
#rslt_df = calendar_df.loc[(calendar_df['listing_id'] == 4592) & (calendar_df['available'] == 'f')]

In [14]:
#print(rslt_df)

In [15]:
#exploratory statistics on each dataset
listings_df.describe()

,id,latitude,longitude,calculated_host_listings_count
count,6.162000e+03,6162.000000,6162.000000,6162.000000
mean,2.086868e+17,37.346771,-121.961575,54.354755
std,3.186442e+17,0.066625,0.111997,130.668417
min,4.952000e+03,36.965600,-122.196520,1.000000
25%,2.599622e+07,37.317750,-122.039510,1.000000
50%,4.625849e+07,37.348895,-121.951896,3.000000
75%,6.028115e+17,37.394208,-121.885593,13.000000
max,7.901690e+17,37.462000,-121.380120,461.000000


In [16]:
#paired with datatype and identification of null values
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6162 non-null   float64
 1   name                            6161 non-null   object 
 2   neighbourhood                   6162 non-null   object 
 3   latitude                        6162 non-null   float64
 4   longitude                       6162 non-null   float64
 5   room_type                       6162 non-null   object 
 6   calculated_host_listings_count  6162 non-null   int64  
 7   license                         258 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 385.2+ KB


In [17]:
#finally check for any duplicates?
listings_df.duplicated().sum()

14

In [18]:
reviews_df.describe() #no staistical value since the rest are boolean or string

,listing_id,review_id
count,1.966170e+05,1.966170e+05
mean,2.610696e+16,2.208022e+17
std,1.263996e+17,2.991275e+17
min,4.952000e+03,6.560000e+03
25%,1.101710e+07,3.223056e+08
50%,2.215150e+07,6.068444e+08
75%,3.818878e+07,5.363594e+17
max,7.803523e+17,7.907592e+17


In [19]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196617 entries, 0 to 196616
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     196617 non-null  int64 
 1   review_id      196617 non-null  int64 
 2   date           196617 non-null  object
 3   reviewer_name  196616 non-null  object
 4   comments       196564 non-null  object
dtypes: int64(2), object(3)
memory usage: 7.5+ MB


In [20]:
reviews_df.duplicated().sum()

0

In [21]:
calendar_df.describe()

,listing_id
count,2.249130e+06
mean,2.086868e+17
std,3.186184e+17
min,4.952000e+03
25%,2.598223e+07
50%,4.625849e+07
75%,6.028160e+17
max,7.901690e+17


In [22]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249130 entries, 0 to 2249129
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   listing_id  int64 
 1   date        object
 2   available   object
dtypes: int64(1), object(2)
memory usage: 51.5+ MB


In [23]:
calendar_df.duplicated().sum()

0

In [24]:
#concentrate on the text cleaning now
#takenfrom https://www.analyticsvidhya.com/blog/2022/01/text-cleaning-methods-in-nlp/
#lowercaseing the data, need as type str so that it processes as string even though datatype is object
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(str.lower)

In [25]:
#test dataframe for testing methods before executing on reviews_df
#reviews_test = reviews_df.copy()

In [26]:
#preview the changes
#reviews_test.head(20)

In [27]:
#remove the <br/> from the reviews
reviews_df['comments'] = reviews_df['comments'].str.replace('<br/>','')

In [28]:
#get rid of the \r
reviews_df['comments'] = reviews_df['comments'].str.replace('\r','')

In [29]:
#remove punctuation
import string
#method
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
#removal from column
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(remove_punctuations)

In [30]:
#remove numbers
reviews_df['comments'] = reviews_df['comments'].str.replace('\d+', '')

<ipython-input-30-ff7a0831e770>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews_df['comments'] = reviews_df['comments'].str.replace('\d+', '')


In [31]:
#remove extra spaces
reviews_df['comments'] = reviews_df['comments'].str.strip()

In [32]:
#remove emojis
import re
#method and library needed to remove emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"U0001F600-U0001F64F"  # emoticons
                           u"U0001F300-U0001F5FF"  # symbols & pictographs
                           u"U0001F680-U0001F6FF"  # transport & map symbols
                           u"U0001F1E0-U0001F1FF"  # flags (iOS)
                           u"U00002702-U000027B0"
                           u"U000024C2-U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [33]:
# actual removal
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(remove_emoji)

In [35]:
#run this each new run then comment it out
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 15.4 MB/s eta 0:00:00


In [36]:
#remove contractions
#pip install library and the method to remove contractions
import contractions
#empty list for expanded text
def remove_contractions(text):
  return contractions.fix(text)

In [37]:
#actually remove
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(remove_contractions)

In [38]:
#remove stopwords, tokenize and create a wordcloud with the words that are most frequent.. later
import nltk
nltk.download('stopwords')
import wordcloud
from nltk import sent_tokenize as st
from nltk import word_tokenize as wt
from nltk.probability import FreqDist as FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

stop_words = nltk.corpus.stopwords.words('english')
print(stop_words)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [39]:
#remove stop words
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
#print to csv for use in tableau and SAS Enterprise Miner
#reviews_df.to_csv("reviewMod.csv")

In [ ]:
#listings_df.to_csv("listingsMod.csv")

In [ ]:
#calendar_df.to_csv("calendarMod.csv")

In [51]:
calendar_df.describe()

,listing_id
count,2.249130e+06
mean,2.086868e+17
std,3.186184e+17
min,4.952000e+03
25%,2.598223e+07
50%,4.625849e+07
75%,6.028160e+17
max,7.901690e+17


In [53]:
calendar_df[['listing_id','available']].value_counts()

listing_id          available
26418356            f            365
48258276            f            365
633452668579962079  t            365
633490375084502061  f            365
633982041953911218  f            365
                                ... 
13599918            f              1
53876775            f              1
9513866             f              1
527119              f              1
51964646            f              1
Length: 11038, dtype: int64

In [1]:
calendar_df.loc[(calendar_df['listing_id'] == 26418356) & (calendar_df['available'] == 'f')]

NameError: ignored